In [1]:
import pandas as pd
df = pd.read_csv("c:/data/customer/customer.csv")
df.columns

Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls',
       'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins',
       'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn'],
      dtype='object')

In [2]:
cols=["Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins',
'Day Calls','Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge',
'Night Mins','Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls',
'Intl Charge','CustServ Calls']
X=df[cols]
y=df['Churn']
y.value_counts()

0    2850
1     483
Name: Churn, dtype: int64

In [3]:
#언더샘플링
from imblearn.under_sampling import RandomUnderSampler
X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)
X_samp = pd.DataFrame(data=X_sample,columns=cols )
y_samp = pd.DataFrame(data=y_sample,columns=['Churn'])
df_samp=pd.concat([X_samp,y_samp],axis=1)
df_samp['Churn'].value_counts()
X=df_samp[cols]
y=df_samp['Churn']

In [9]:
#시간이 오래 걸림
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
# 선형모형만 가능함(rbf 커널에는 coef가 제공되지 않음)
svc = SVC(kernel="linear")
# step=1 각 단계에서 제거할 변수의 개수
rfecv = RFECV(estimator=svc, step=2, cv=StratifiedKFold(3), scoring='accuracy')
# logit/tree/svc(linear)만 사용가능
# step: 단계별 감소할 변수
rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
      estimator=SVC(kernel='linear'), scoring='accuracy', step=2)

In [11]:
rfecv.cv_results_['mean_test_score']

array([0.60973085, 0.6252588 , 0.69151139, 0.72670807, 0.74223602,
       0.76086957, 0.76190476, 0.76190476, 0.76501035])

In [15]:
import numpy as np
print(np.mean(rfecv.grid_scores_,axis=1)) #교차검증으로 계산한 변수들의 중요도
print(rfecv.n_features_) #변수 개수
print(rfecv.support_) #변수 선택 여부
print(rfecv.ranking_) #변수 선택 랭킹
#선택된 변수들
print(X.columns[rfecv.support_])
#선택되지 않은 변수들
print(X.columns.difference(X.columns[rfecv.support_]))
df2=pd.DataFrame(list(zip(X.columns,np.mean(rfecv.grid_scores_,axis=1))),columns=['변수명','평균점수'])
df2

[0.60973085 0.6252588  0.69151139 0.72670807 0.74223602 0.76086957
 0.76190476 0.76190476 0.76501035]
16
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls',
       'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins',
       'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls'],
      dtype='object')
Index([], dtype='object')


c:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: The `grid_scores_` attribute is deprecated in version 1.0 in favor of `cv_results_` and will be removed in version 1.2.
  warnings.warn(msg, category=FutureWarning)


TypeError: list() takes no keyword arguments

In [6]:
df2.sort_values(by='평균점수', ascending=False)

,변수명,평균점수
9,Night Mins,0.768121
13,Intl Calls,0.766038
12,Intl Mins,0.766033
10,Night Calls,0.765007
14,Intl Charge,0.765007
15,CustServ Calls,0.765007
11,Night Charge,0.759831
8,Eve Charge,0.751541
7,Eve Calls,0.748427
6,Eve Mins,0.748422
